In [ ]:
# Get necessary packages
import numpy as np
import pandas as pd
from scipy.io import *
from pylab import *
from pandas import DataFrame as df
from hmmlearn import hmm
# import random
from scipy import stats
from scipy.spatial import distance

In [ ]:
#Retrieve FPP and community clusters
FPPs = loadmat('Matrix120_OR.mat')['s'][0]
communitiest = loadmat('Cluster120_OR.mat')['b'][0]
# mod = loadmat('structure_stats.mat')['c'][0]

In [ ]:
def Markov_Clustering_Gauss(FPP, c):
    '''
    Get markov clusters for gamma only works for clusters with probabilities adding up to 1
    Input: theta cycles matrix, inflation parameter
    Output: labels for each FPP
    '''
    gm =hmm.GaussianHMM(c)
    #get parameters
    model =gm.fit(theta, len(FPP))
    clusters = gm.predict(FPP)              # get clusters
    return clusters

In [ ]:
def Markov_Clustering_Gmm(FPP, c):
    '''
    Get markov clusters for gamma
    Input: theta cycles matrix, inflation parameter
    Output: labels for each FPP
    '''
    gm =hmm.GMMHMM(c)
    # get parameters
    model =gm.fit(FPP, len(FPP))
    # get clusters
    clusters = gm.predict(FPP)
    return clusters

In [ ]:
#  Get amount of clusters expected through community clustering
PFC_means=[]
for i in range(1,16, 2):
    PFC_means.append(np.mean(communitiest[i]))
PFC_k= round(np.mean(np.array(PFC_means)))
HPC_means=[]
for i in range(0,16,2):
    HPC_means.append(np.mean(communitiest[i]))
HPC_k=round(np.mean(np.array(HPC_means)))


In [ ]:
# Retrieve HMM clustering labels
pfc_clusters=[]
for i in range(1,16,2):
    rat = FPPs[i]
    c = PFC_k
    pfc_clusters.append(Markov_Clustering_Gmm(rat, c))
hpc_clusters=[]
for i in range(0,16,2):
    rat = FPPs[i]
    c = HPC_k
    hpc_clusters.append(Markov_Clustering_Gmm(rat, c))
    

In [ ]:
def recluster(FPP, clusters, start=0, step=2):
    '''
    Input: all FPPs, cluster labels from kmeans, start and step point rat iteration
    Export: all clusters for all rats
    Retrieve the data within the clusters
    '''
    pls = []
    for i in range(start,16,step):
        cl_ = []
        clust = dict()
        for j in range(0,len(np.array(clusters[i//2]))):
            clus=np.array(clusters[i//2])[j]
            cls_=[]
            F= FPP[i]
            cluster=F[j]
            if clus not in clust:
                clust[clus]=np.array([cluster])
            else:
                clust[clus]= np.append(clust[clus],[cluster], axis=0)
        pls.append([clust])
    return pls

In [ ]:
pls_pfc = recluster(FPPs, pfc_clusters, 1)
pls_hpc = recluster(FPPs, hpc_clusters)

In [ ]:
def pearsoncor(x, y):
    r = stats.pearsonr(x, y)[0]
    return r
def pearson_dist(x, y):
    r = stats.pearsonr(x, y)[0]
    return (1 - r) / 2

def mahalanobis_dist(x, y, lr=False):
    m =list(zip(x, y))
    try:
        iv = np.linalg.inv(np.cov(m))
    except:
        iv = np.linalg.pinv(np.cov(m)) 
    return distance.mahalanobis(x, y,iv)
def lrratio(x):
    l=(stats.norm.cdf(np.array(x)**2))
    return (1-l)/len(x)
def intracor(clusters, FPP_k, pearsoncor=False, mahalanobis=False):
    '''
    Retrieve correlations between the points in a specific cluster
    Input: Singular FPP, mean of all FPP within cluster, distance function indicator
    '''
    intra = []
    for j in range(len(clusters)):
        FPP_i=clusters[j]
        if mahalanobis:
            intra.append(mahalanobis_dist(FPP_i, FPP_k))
        elif pearsoncor:
            intra.append(pearsoncor(FPP_i, FPP_k))
        #Default pearson cor
        else:
            intra.append(pearson_dist(FPP_i, FPP_k))
    return intra

def extracor(clusters, FPP_k, pearsoncor=False,lr=False, mahalanobis=False):
    '''
    Retrieve correlations between the points in a specific cluster and the other clusters
    Input: Singular FPP, mean of all FPP outside of clusters, distance function indicator
    '''
    extra=[]
    FPP_k= np.mean(FPP_k, axis=0)
    for j in range(len(clusters)):
        FPP_i=clusters[j]
        if mahalanobis or lr:
            extra.append(mahalanobis_dist(FPP_i, FPP_k))
        elif pearsoncor:
            extra.append(pearsoncor(FPP_i, FPP_k))
        #Default pearson cor
        else:
            extra.append(pearson_dist(FPP_i, FPP_k))
    if lr:
        return np.nanmean(lrratio(extra))
    return np.nanmean(extra)

In [ ]:
def gamma_sort(pls,num,region, threshold = 0.95):
    '''
    Input: clusters for all rats, number of clusters, region and threshold for centroid of gamma
    Output:mean (res/results) of all FPP in clusters for all rats, significancies for all distance measures  
    Sort into gamma clusters, retrieve significance and plot
    note: when num>1 iterate over ex_clusters
    '''
    res = []
    accuracy_ex=[]
    accuracy_in=[]
    M_accuracy_ex=[]
    M_accuracy_in=[]
    lr_accuracy =[]
    for j in range(len(pls)):
        print('Rat')
        cls_=pls[j][0]
        clusters=[]
        ex_clusters= cls_.copy()
        in_acc=[]
        ex_acc=[]
        lr_acc=[]
        M_ex_clusters= cls_.copy()
        M_in_acc=[]
        M_ex_acc=[]
        lr_ex_clusters= cls_.copy()
        for i in range(num):
            print('Cluster')
            cluster_init =np.array(cls_[i])

#             gamma_peak = threshold*max(np.array([np.max(c) for c in cluster]))
            cluster=cluster_init.reshape(len(cluster_init),51,20)
            masked_cluster=np.mean(cluster, axis=0)
            #Measure significance
            
            ex_clusters.pop(i)
            for c in list(ex_clusters.keys()):
                #Pearson
                intra = np.nanmean(intracor(cluster_init, np.mean(cluster_init, axis=0)))
                extra=extracor(cluster_init, np.array(ex_clusters[c]))
                in_acc.append(intra)
                ex_acc.append(extra)
                ex_clusters=cls_.copy()
                #mahalanobis
                M_ex_clusters.pop(i)
                M_intra = np.nanmean(intracor(cluster_init, np.mean(cluster_init, axis=0), mahalanobis=True))
                M_extra=extracor(cluster_init, np.array(M_ex_clusters[c]), mahalanobis=True)
                M_in_acc.append(M_intra)
                M_ex_acc.append(M_extra)
                M_ex_clusters=cls_.copy()
                #Lratio
#                 lr_ex_clusters.pop(i)
#                 lr=extracor(masked_cluster, np.array(lr_ex_clusters[c]),lr=True)
#                 lr_acc.append(lr)
#                 lr_ex_clusters=cls_.copy()

            
            imshow(masked_cluster,extent=[-np.pi, np.pi, 120, 20],aspect='auto', cmap = 'hot' )
            ylabel('Frequencies(Hz)')    
            xlabel('Phase')
            colorbar()
            if j>=4:
                title('Rat %i cluster %i %s'%(j+2,i+1, region))

                savefig('freq120/hmm/HMM Rat %i cluster %i %s'%(j+2,i+1, region))
            else:
                title('Rat %i cluster %i %s'%(j+1,i+1, region))
                savefig('freq120/hmm/HMM Rat %i cluster %i %s'%(j+1,i+1, region))
            show()
            #show average bin
#             for c in range(20):
#                 imshow(np.array([masked_cluster[:,c]]).T,aspect='auto' )
#                 ylabel('Frequencies(Hz)')
#                 colorbar()
#                 xlabel('Phase')
#                 if j>=4:
#                     title('Rat %i cluster %i cycle %i'%(j+2,i+1, c+1 ))

#     #                 savefig('Rat %i cluster %i %s cycle %i'%(j+2,i+1, region, c+1))
#                 else:
#                     title('Rat %i cluster %i cycle %i'%(j+1,i+1, c+1))
#     #                 savefig('Rat %i cluster %i %s cycle %i'%(j+1,i+1, region, c+1)
#                 show()
#             print(list(range(len(cluster))))
#             l=random.choices(list(range(len(cluster))), k=3)
#             for f in l:
#                 imshow(cluster[f],extent=[-np.pi, np.pi, 120, 20],aspect='auto' )
#                 ylabel('Frequencies(Hz)')
#                 colorbar()
#                 xlabel('Phase')
#                 if j>=4:
#                     title('Rat %i cluster %i random sample %s'%(j+2,c+1, region))

#                     savefig('HMM Rat %i cluster %i random sample %i'%(j+2,c+1, f+1))
#                 else:
#                     title('Rat %i cluster %i random sample %s'%(j+1,c+1, region))
#                     savefig('HMM Rat %i cluster %i random sample %i'%(j+1,c+1, f+1))
#                 show()
            
            clusters.append(masked_cluster)
        accuracy_ex.append(ex_acc)
        accuracy_in.append(in_acc)
        M_accuracy_ex.append(M_ex_acc)
        M_accuracy_in.append(M_in_acc)
#         lr_accuracy.append(lr_acc)
        res.append(clusters)
    return res, accuracy_ex, accuracy_in, M_accuracy_ex,M_accuracy_in

In [ ]:
gamma_clusters_pfc,pfc_cluster_ex, pfc_cluster_in, mpfc_cluster_ex, mpfc_cluster_in=gamma_sort(pls_pfc, 2, 'PFC')
gamma_clusters_hpc,hpc_cluster_ex,hpc_cluster_in, mhpc_cluster_ex, mhpc_cluster_in=gamma_sort(pls_hpc, 2, 'HPC')

In [ ]:
# Store all data in dataframe
df_clusters =  df(columns =['Rat', 'Cluster_Number','Mutation/Control', 'Condition','PFC_Clusters', 'HPC_Clusters', 'PFC_Gamma_Clusters', 'HPC_Gamma_Clusters'])
df_clusters['PFC_Clusters']= df_clusters['PFC_Clusters'].astype(object)
df_clusters['HPC_Clusters']= df_clusters['HPC_Clusters'].astype(object)
df_clusters['PFC_Gamma_Clusters']= df_clusters['PFC_Gamma_Clusters'].astype(object)
df_clusters['HPC_Gamma_Clusters']= df_clusters['HPC_Gamma_Clusters'].astype(object)
mclist=['Control', 'Control','Mutation', 'Mutation',  'Control', 'Mutation', 'Mutation','Control']
for i in range(0,8):
    j=i+1   
    if j >= 5:
        j+=1
    for num in range(len(gamma_clusters_hpc[i])):
        hpc_extra = hpc_cluster_ex[i][num]
        hpc_intra =hpc_cluster_in[i][num]
        pfc_extra =pfc_cluster_ex[i][num]
        pfc_intra = pfc_cluster_in[i][num]
        m_hpc_extra = mhpc_cluster_ex[i][num]
        m_hpc_intra =mhpc_cluster_in[i][num]
        m_pfc_extra =mpfc_cluster_ex[i][num]
        m_pfc_intra = mpfc_cluster_in[i][num]
#         lr_hpc_extra = lr_hpc[i][num]
#         lr_pfc_extra =lr_pfc[i][num]
        df_clusters=df_clusters.append({'Rat':j,'Cluster_Number':num+1,'Mutation/Control': mclist[i],'Condition':'Overlapping','PearsonDistance_PFC_Intra_Cluster': pfc_intra,\
                                        'PearsonDistance_PFC_Inter_cluster':pfc_extra,'PearsonDistance_HPC_Intra_Cluster': hpc_intra, \
                                        'PearsonDistance_HPC_Inter_cluster':hpc_extra,'MahalanobisDistance_PFC_Intra_Cluster': m_pfc_intra,\
                                        'MahalanobisDistance_PFC_Inter_cluster':m_pfc_extra,'MahalanobisDistance_HPC_Intra_Cluster': m_hpc_intra, \
                                        'MahalanobisDistance_HPC_Inter_cluster':m_hpc_extra,'PFC_Clusters':pfc_clusters[i], 'HPC_Clusters': hpc_clusters[i], \
                                        'PFC_Gamma_Clusters':gamma_clusters_pfc[i][num], 'HPC_Gamma_Clusters':gamma_clusters_hpc[i][num]}, ignore_index=True)

In [ ]:
df_clusters.to_csv('../Clusters/OverlappingHMMMMeanClusters/Clustershmm_120_2_overlapping.csv',index=False)

In [ ]:
df_clusters